<a href="https://colab.research.google.com/github/leahandofir/earthformer-inference-experiments/blob/main/earthformer_inference_experiment_without_conda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Genral things I did:
**The goal** - try to understand what we need to do to run model(x) and get a result. **STILL DID NOT REACHED THAT GOAL**
for now, tried to solve the error `ImportError: cannot import name '_PATH' from 'pytorch_lightning.utilities.types`
with some google advices with no success.
after that saw this error in the pip outputs:
```
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 1.12.1+cu116 which is incompatible.
```
also this:
```
ERROR: pip's dependency resolver does not currently 
take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.1+cu118 requires torch==2.0.0, but you have torch 1.12.1+cu116 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.12.1+cu116 which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 1.12.1+cu116 which is incompatible.
```
so decided to try and install everything with cuda 11.8 and let pip decide the versions.
after that it just worked.
general good things to copy from their code:
- they use os.path a lot. checking if directories exist, and if not they create them. we need our code to be as little as possible dependent on the machine.
- they have utils directory of all the general needed things (like download a given url)

In [ ]:
%%shell
# install dependencies
pip install --upgrade pip
python3 -m pip install torch==2.0.0+cu118 torchvision -f https://download.pytorch.org/whl/torch_stable.html
python3 -m pip install "pytorch_lightning>=1.6.4,<1.8.0"
python3 -m pip install xarray netcdf4 opencv-python
git clone https://github.com/amazon-science/earth-forecasting-transformer.git
cd earth-forecasting-transformer
python3 -m pip install -U -e . --no-build-isolation
python3 -m pip install setuptools
python3 -m pip install packaging
pip install --upgrade setuptools wheel Cmake numpy numba ninja
pip install -v --disable-pip-version-check --no-cache-dir pytorch-extension git+https://github.com/NVIDIA/apex.git

In [ ]:
%cd /content/earth-forecasting-transformer/src

/content/earth-forecasting-transformer/src


In [ ]:
import warnings
import os
from omegaconf import OmegaConf
import pytest
import numpy as np
import torch
from torch.nn import functional as F
import torchmetrics
from einops import rearrange
from earthformer.config import cfg
from earthformer.utils.checkpoint import s3_download_pretrained_ckpt
from earthformer.utils.layout import layout_to_in_out_slice
from earthformer.utils.utils import download
from earthformer.cuboid_transformer.cuboid_transformer import CuboidTransformerModel
from earthformer.cuboid_transformer.cuboid_transformer_unet_dec import CuboidTransformerAuxModel


NUM_TEST_ITER = 16  # max = 32 since saved `unittest_data.pt` only contains the first 0 to 31 data entries.
test_data_dir = os.path.join(cfg.root_dir, "tests", "unittests", "test_pretrained_checkpoints_data")

def s3_download_unittest_data(data_name):
    test_data_path = os.path.join(test_data_dir, data_name)
    if not os.path.exists(test_data_path):
        os.makedirs(test_data_dir, exist_ok=True)
        download(url=f"s3://deep-earth/experiments/earthformer/unittests/{data_name}", path=test_data_path)


def config_cuboid_transformer(cfg, model_type="CuboidTransformerModel"):
    model_cfg = OmegaConf.to_object(cfg.model)
    num_blocks = len(model_cfg["enc_depth"])
    if isinstance(model_cfg["self_pattern"], str):
        enc_attn_patterns = [model_cfg.pop("self_pattern")] * num_blocks
    else:
        enc_attn_patterns = OmegaConf.to_container(model_cfg.pop("self_pattern"))
    model_cfg["enc_attn_patterns"] = enc_attn_patterns
    if isinstance(model_cfg["cross_self_pattern"], str):
        dec_self_attn_patterns = [model_cfg.pop("cross_self_pattern")] * num_blocks
    else:
        dec_self_attn_patterns = OmegaConf.to_container(model_cfg.pop("cross_self_pattern"))
    model_cfg["dec_self_attn_patterns"] = dec_self_attn_patterns
    if isinstance(model_cfg["cross_pattern"], str):
        dec_cross_attn_patterns = [model_cfg.pop("cross_pattern")] * num_blocks
    else:
        dec_cross_attn_patterns = OmegaConf.to_container(model_cfg.pop("cross_pattern"))
    model_cfg["dec_cross_attn_patterns"] = dec_cross_attn_patterns
    if model_type == "CuboidTransformerModel":
        model = CuboidTransformerModel(**model_cfg)
    elif model_type == "CuboidTransformerAuxModel":
        model = CuboidTransformerAuxModel(**model_cfg)
    else:
        raise ValueError(f"Invalid model_type {model_type}. Must be 'CuboidTransformerModel' or ''.")
    return model

def test_sevir():
    pretrained_ckpt_name = "earthformer_sevir.pt"
    test_data_name = "unittest_sevir_data_bs1_idx0to31.pt"
    s3_download_unittest_data(data_name=test_data_name)
    test_data_path = os.path.join(test_data_dir, test_data_name)
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    # Load pretrained model
    pretrained_cfg_path = os.path.join(cfg.root_dir, "scripts", "cuboid_transformer", "sevir", "earthformer_sevir_v1.yaml")
    pretrained_cfg = OmegaConf.load(open(pretrained_cfg_path, "r"))
    model = config_cuboid_transformer(
        cfg=pretrained_cfg,
        model_type="CuboidTransformerModel").to(device)
    model.eval()
    if not os.path.exists(os.path.join(cfg.pretrained_checkpoints_dir, pretrained_ckpt_name)):
        s3_download_pretrained_ckpt(ckpt_name=pretrained_ckpt_name,
                                    save_dir=cfg.pretrained_checkpoints_dir,
                                    exist_ok=False)
    state_dict = torch.load(os.path.join(cfg.pretrained_checkpoints_dir, pretrained_ckpt_name),
                            map_location=device)
    missing_keys, unexpected_keys = model.load_state_dict(state_dict=state_dict, strict=False)
    assert len(missing_keys) == 0, f"missing_keys {missing_keys} when loading pretrained state_dict."
    assert len(unexpected_keys) == 0, f"missing_keys {unexpected_keys} when loading pretrained state_dict."
    # Test on SEVIR test
    layout_cfg = pretrained_cfg.layout
    in_slice, out_slice = layout_to_in_out_slice(layout=layout_cfg.layout,
                                                 in_len=layout_cfg.in_len,
                                                 out_len=layout_cfg.out_len)
    test_mse_metrics = torchmetrics.MeanSquaredError().to(device)
    test_mae_metrics = torchmetrics.MeanAbsoluteError().to(device)
    test_data = torch.load(test_data_path)
    counter = 0
    with torch.no_grad():
        for batch in test_data:
            data_seq = batch['vil'].contiguous().to(device)
            x = data_seq[in_slice]
            y = data_seq[out_slice]
            y_hat = model(x)
            test_mse_metrics(y_hat, y)
            test_mae_metrics(y_hat, y)
            counter += 1
            if counter >= NUM_TEST_ITER:
                break
    test_mse = test_mse_metrics.compute()
    test_mae = test_mae_metrics.compute()
    assert test_mse < 1E-2
    assert test_mae < 5E-2

def test_enso():
    pretrained_ckpt_name = "earthformer_icarenso2021.pt"
    test_data_name = "unittest_icarenso2021_data_bs1_idx0to31.pt"
    s3_download_unittest_data(data_name=test_data_name)
    test_data_path = os.path.join(test_data_dir, test_data_name)
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    # Load pretrained model
    pretrained_cfg_path = os.path.join(cfg.root_dir, "scripts", "cuboid_transformer", "enso", "earthformer_enso_v1.yaml")
    pretrained_cfg = OmegaConf.load(open(pretrained_cfg_path, "r"))
    model = config_cuboid_transformer(
        cfg=pretrained_cfg,
        model_type="CuboidTransformerModel").to(device)
    model.eval()

    if not os.path.exists(os.path.join(cfg.pretrained_checkpoints_dir, pretrained_ckpt_name)):
        s3_download_pretrained_ckpt(ckpt_name=pretrained_ckpt_name,
                                    save_dir=cfg.pretrained_checkpoints_dir,
                                    exist_ok=False)
    state_dict = torch.load(os.path.join(cfg.pretrained_checkpoints_dir, pretrained_ckpt_name),
                            map_location=device)
    missing_keys, unexpected_keys = model.load_state_dict(state_dict=state_dict, strict=False)
    assert len(missing_keys) == 0, f"missing_keys {missing_keys} when loading pretrained state_dict."
    assert len(unexpected_keys) == 0, f"missing_keys {unexpected_keys} when loading pretrained state_dict."
    # Test on ENSO test
    layout_cfg = pretrained_cfg.layout
    in_slice, out_slice = layout_to_in_out_slice(layout=layout_cfg.layout,
                                                 in_len=layout_cfg.in_len,
                                                 out_len=layout_cfg.out_len)
    test_mse_metrics = torchmetrics.MeanSquaredError().to(device)
    test_mae_metrics = torchmetrics.MeanAbsoluteError().to(device)
    test_data = torch.load(test_data_path)
    counter = 0
    with torch.no_grad():
        for batch in test_data:
            sst_seq, nino_target = batch
            data_seq = sst_seq.float().unsqueeze(-1).to(device)
            x = data_seq[in_slice]
            y = data_seq[out_slice]
            y_hat = model(x)
            test_mse_metrics(y_hat, y)
            test_mae_metrics(y_hat, y)
            counter += 1
            if counter >= NUM_TEST_ITER:
                break
    test_mse = test_mse_metrics.compute()
    test_mae = test_mae_metrics.compute()
    assert test_mse < 5E-4
    assert test_mae < 2E-2

def test_earthnet():
    data_channels = 4
    pretrained_ckpt_name = "earthformer_earthnet2021.pt"
    test_data_name = "unittest_earthnet2021_data_bs1_idx0to31.pt"
    s3_download_unittest_data(data_name=test_data_name)
    test_data_path = os.path.join(test_data_dir, test_data_name)
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    # Load pretrained model
    pretrained_cfg_path = os.path.join(cfg.root_dir, "scripts", "cuboid_transformer", "earthnet_w_meso", "earthformer_earthnet_v1.yaml")
    pretrained_cfg = OmegaConf.load(open(pretrained_cfg_path, "r"))
    model = config_cuboid_transformer(
        cfg=pretrained_cfg,
        model_type="CuboidTransformerAuxModel").to(device)
    model.eval()

    if not os.path.exists(os.path.join(cfg.pretrained_checkpoints_dir, pretrained_ckpt_name)):
        s3_download_pretrained_ckpt(ckpt_name=pretrained_ckpt_name,
                                    save_dir=cfg.pretrained_checkpoints_dir,
                                    exist_ok=False)
    state_dict = torch.load(os.path.join(cfg.pretrained_checkpoints_dir, pretrained_ckpt_name),
                            map_location=device)
    missing_keys, unexpected_keys = model.load_state_dict(state_dict=state_dict, strict=False)
    assert len(missing_keys) == 0, f"missing_keys {missing_keys} when loading pretrained state_dict."
    assert len(unexpected_keys) == 0, f"missing_keys {unexpected_keys} when loading pretrained state_dict."
    # Test on EarthNet2021 test
    layout_cfg = pretrained_cfg.layout
    in_slice, out_slice = layout_to_in_out_slice(layout=layout_cfg.layout,
                                                 in_len=layout_cfg.in_len,
                                                 out_len=layout_cfg.out_len)
    test_mse_metrics = torchmetrics.MeanSquaredError().to(device)
    test_mae_metrics = torchmetrics.MeanAbsoluteError().to(device)
    test_data = torch.load(test_data_path)
    counter = 0
    with torch.no_grad():
        for batch in test_data:
            highresdynamic = batch["highresdynamic"].to(device)
            seq = highresdynamic[..., :data_channels]
            print(seq.shape)
            # mask from dataloader: 1 for mask and 0 for non-masked
            mask = highresdynamic[..., data_channels: data_channels + 1][out_slice]
            in_seq = seq[in_slice]
            target_seq = seq[out_slice]
            # process aux data
            highresstatic = batch["highresstatic"].to(device)  # (b c h w)
            mesodynamic = batch["mesodynamic"].to(device)  # (b t h w c)
            mesostatic = batch["mesostatic"].to(device)  # (b c h w)
            mesodynamic_interp = rearrange(mesodynamic,
                                           "b t h w c -> b c t h w")
            mesodynamic_interp = F.interpolate(mesodynamic_interp,
                                               size=(layout_cfg.in_len + layout_cfg.out_len,
                                                     layout_cfg.img_height,
                                                     layout_cfg.img_width),
                                               mode="nearest")
            highresstatic_interp = rearrange(highresstatic,
                                             "b c h w -> b c 1 h w")
            highresstatic_interp = F.interpolate(highresstatic_interp,
                                                 size=(layout_cfg.in_len + layout_cfg.out_len,
                                                       layout_cfg.img_height,
                                                       layout_cfg.img_width),
                                                 mode="nearest")
            mesostatic_interp = rearrange(mesostatic,
                                          "b c h w -> b c 1 h w")
            mesostatic_interp = F.interpolate(mesostatic_interp,
                                              size=(layout_cfg.in_len + layout_cfg.out_len,
                                                    layout_cfg.img_height,
                                                    layout_cfg.img_width),
                                              mode="nearest")
            aux_data = torch.cat((highresstatic_interp, mesodynamic_interp, mesostatic_interp),
                                 dim=1)
            aux_data = rearrange(aux_data,
                                 "b c t h w -> b t h w c")
            pred_seq = model(in_seq, aux_data[in_slice], aux_data[out_slice])
            test_mse_metrics(pred_seq * (1 - mask), target_seq * (1 - mask))
            test_mae_metrics(pred_seq * (1 - mask), target_seq * (1 - mask))
            counter += 1
            if counter >= NUM_TEST_ITER:
                break
    test_mse = test_mse_metrics.compute()
    test_mae = test_mae_metrics.compute()
    assert test_mse < 5E-4
    assert test_mae < 1E-2

In [ ]:
  # trying to run some parts to understand what is going on
    pretrained_ckpt_name = "earthformer_sevir.pt"
    test_data_name = "unittest_sevir_data_bs1_idx0to31.pt"
    s3_download_unittest_data(data_name=test_data_name)
    test_data_path = os.path.join(test_data_dir, test_data_name)
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    # Load pretrained model
    pretrained_cfg_path = os.path.join(cfg.root_dir, "scripts", "cuboid_transformer", "sevir", "earthformer_sevir_v1.yaml")
    pretrained_cfg = OmegaConf.load(open(pretrained_cfg_path, "r"))
    model = config_cuboid_transformer(
        cfg=pretrained_cfg,
        model_type="CuboidTransformerModel").to(device)
    model.eval()

In [ ]:
model # holds all layers of the model

CuboidTransformerModel(
  (initial_encoder): InitialStackPatchMergingEncoder(
    (conv_block_list): ModuleList(
      (0): Sequential(
        (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): GroupNorm(4, 16, eps=1e-05, affine=True)
        (2): LeakyReLU(negative_slope=0.1)
        (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): GroupNorm(4, 16, eps=1e-05, affine=True)
        (5): LeakyReLU(negative_slope=0.1)
      )
      (1): Sequential(
        (0): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): GroupNorm(16, 64, eps=1e-05, affine=True)
        (2): LeakyReLU(negative_slope=0.1)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): GroupNorm(16, 64, eps=1e-05, affine=True)
        (5): LeakyReLU(negative_slope=0.1)
      )
      (2): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): Grou

In [ ]:
    if not os.path.exists(os.path.join(cfg.pretrained_checkpoints_dir, pretrained_ckpt_name)):
        s3_download_pretrained_ckpt(ckpt_name=pretrained_ckpt_name,
                                    save_dir=cfg.pretrained_checkpoints_dir,
                                    exist_ok=False)
    state_dict = torch.load(os.path.join(cfg.pretrained_checkpoints_dir, pretrained_ckpt_name),
                            map_location=device)
    missing_keys, unexpected_keys = model.load_state_dict(state_dict=state_dict, strict=False)
    assert len(missing_keys) == 0, f"missing_keys {missing_keys} when loading pretrained state_dict."
    assert len(unexpected_keys) == 0, f"missing_keys {unexpected_keys} when loading pretrained state_dict."
    # Test on SEVIR test
    layout_cfg = pretrained_cfg.layout
    in_slice, out_slice = layout_to_in_out_slice(layout=layout_cfg.layout,
                                                 in_len=layout_cfg.in_len,
                                                 out_len=layout_cfg.out_len)
    test_mse_metrics = torchmetrics.MeanSquaredError().to(device)
    test_mae_metrics = torchmetrics.MeanAbsoluteError().to(device)
    test_data = torch.load(test_data_path)
    counter = 0

In [ ]:
layout_cfg

{'in_len': 13, 'out_len': 12, 'layout': 'NTHWC'}

In [ ]:
test_data # this is data!!! :O how can we see the pictures?!

[{'vil': tensor([[[[[0.0000],
             [0.0000],
             [0.0000],
             ...,
             [0.0000],
             [0.0000],
             [0.0000]],
  
            [[0.0000],
             [0.0000],
             [0.0000],
             ...,
             [0.0000],
             [0.0000],
             [0.0000]],
  
            [[0.0000],
             [0.0000],
             [0.0000],
             ...,
             [0.0000],
             [0.0000],
             [0.0000]],
  
            ...,
  
            [[0.0000],
             [0.0000],
             [0.0000],
             ...,
             [0.0000],
             [0.0000],
             [0.0000]],
  
            [[0.0000],
             [0.0000],
             [0.0000],
             ...,
             [0.0000],
             [0.0000],
             [0.0000]],
  
            [[0.0000],
             [0.0000],
             [0.0000],
             ...,
             [0.0000],
             [0.0000],
             [0.0000]]],
  
  
         

In [8]:
%%shell
# install aws cli to check the size of the sevir dataset
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip awscliv2.zip
sudo ./aws/install

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 54.9M  100 54.9M    0     0   156M      0 --:--:-- --:--:-- --:--:--  162M
Archive:  awscliv2.zip
replace aws/THIRD_PARTY_LICENSES? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: aws/THIRD_PARTY_LICENSES  
replace aws/README.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: Found preexisting AWS CLI installation: /usr/local/aws-cli/v2/current. Please rerun install script with --update flag.


CalledProcessError: ignored

In [ ]:
%%shell
# checking the aws cli installation and the size of the sevir dataset
eval "$(conda shell.bash hook)" # copy conda command to shell
conda activate earthformer
aws --version
aws s3 ls --no-sign-request s3://sevir/data/vil --recursive --summarize --human-readable